In [3]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os, sys
project_root = os.path.abspath('/Users/subhojit/workspace/saturn/src')
if project_root not in sys.path:
    sys.path.append(project_root)

from reward_model import RewardModel
from dataloader import PreferenceDatasetLite
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def pairwise_reward_loss(chosen_rewards, rejected_rewards):
    return -torch.nn.functional.logsigmoid(chosen_rewards - rejected_rewards).mean()

In [7]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
data = []
dataset = PreferenceDatasetLite(data, tokenizer)
loader = DataLoader(dataset, batch_size=1)

model = RewardModel('gpt2')
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

for epoch in range(3):
    for chosen, rejected in loader:
        chosen_reward = model(**{k: v.squeeze(0) for k, v in chosen.items()})
        rejected_reward = model(**{k: v.squeeze(0) for k, v in rejected.items()})

        loss = pairwise_reward_loss(chosen_reward, rejected_reward)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
